In [2]:
import pandas as pd

# Load the two CSV files for comparison
file1_df = pd.read_csv('filtered_file1.csv')
file2_df = pd.read_csv('filtered_file2.csv')

# Define the column pairs to compare
columns_to_compare = [
    ("Sellers", "Sellers"),
    ("Buyers", "Buyers"),
    ("Brokers", "Brokers"),
    ("Surgery", "Surgery")
]

columns_to_compare2 = [
    ("Sellers_x", "Sellers_y"),
    ("Buyers_x", "Buyers_y"),
    ("Brokers_x", "Brokers_y"),
    ("Surgery_x", "Surgery_y")
]

# Mapping of common abbreviations to full country names
country_mapping = {
    'UK': 'United Kingdom',
    'US': 'United States',
    'England': 'United Kingdom',
    'USA': 'United States',
}

# Function to replace abbreviations with full country names
def replace_abbreviations(cell):
    if pd.isna(cell):
        return cell
    for abbr, full_name in country_mapping.items():
        cell = str(cell).replace(abbr, full_name)
    return cell

# Apply the function to relevant columns in both dataframes
for col1, _ in columns_to_compare:
    file1_df[col1] = file1_df[col1].apply(replace_abbreviations)
for _, col2 in columns_to_compare:
    file2_df[col2] = file2_df[col2].apply(replace_abbreviations)

# Function to compare two cells
def cell_comparison(cell1, cell2):
    set1 = set(str(cell1).split(', '))
    set2 = set(str(cell2).split(', '))
    return not set1.isdisjoint(set2)

# Merge the two dataframes on ID
merged_df = pd.merge(file1_df, file2_df, on="ID")

# Print match counts for each column pair
for col1, col2 in columns_to_compare2:
    comparison = merged_df.apply(lambda row: cell_comparison(row[col1], row[col2]), axis=1)
    match_count = comparison.sum()
    total_count = len(comparison)
    print(f"{col1}-{col2}: {match_count}/{total_count} matches ({(match_count / total_count) * 100:.2f}%)")


Sellers_x-Sellers_y: 118/136 matches (86.76%)
Buyers_x-Buyers_y: 104/136 matches (76.47%)
Brokers_x-Brokers_y: 100/136 matches (73.53%)
Surgery_x-Surgery_y: 115/136 matches (84.56%)


In [1]:
import pandas as pd

# Load the uploaded CSV files
file1_path = 'filtered_file1.csv'
file2_path = 'filtered_file2.csv'

file1_df = pd.read_csv(file1_path)
file2_df = pd.read_csv(file2_path)

# Define the columns to compare
columns_to_compare = [
    ("Sellers", "Sellers"),
    ("Buyers", "Buyers"),
    ("Brokers", "Brokers"),
    ("Surgery", "Surgery")
]

# Mapping of common abbreviations to full country names
country_mapping = {
    'UK': 'United Kingdom',
    'US': 'United States',
    'England': 'United Kingdom',
    'USA': 'United States',
}

# Function to replace abbreviations with full country names
def replace_abbreviations(cell):
    if pd.isna(cell):
        return cell
    for abbr, full_name in country_mapping.items():
        cell = str(cell).replace(abbr, full_name)
    return cell

# Apply the function to relevant columns in both dataframes
for col1, _ in columns_to_compare:
    file1_df[col1] = file1_df[col1].apply(replace_abbreviations)
    file2_df[col1] = file2_df[col1].apply(replace_abbreviations)

# Merge the two dataframes on ID
merged_df = pd.merge(file1_df, file2_df, on="ID", suffixes=('_x', '_y'))

# Function to calculate precision and recall
def precision_recall(true_str, pred_str):
    if pd.isna(true_str) and pd.isna(pred_str):
        return 1.0, 1.0  # Both empty
    if pd.isna(true_str):
        return 0.0, 1.0  # Nothing in ground truth, all prediction is FP
    if pd.isna(pred_str):
        return 1.0, 0.0  # All ground truth missed

    true_set = set(map(str.strip, str(true_str).split(',') if true_str else []))
    pred_set = set(map(str.strip, str(pred_str).split(',') if pred_str else []))
    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    return precision, recall

# Calculate average precision and recall per column
results = {}
for col in ['Sellers', 'Buyers', 'Brokers', 'Surgery']:
    metrics = merged_df.apply(lambda row: precision_recall(row[f"{col}_x"], row[f"{col}_y"]), axis=1)
    precisions, recalls = zip(*metrics)

    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)

    # F1 Score: harmonic mean of precision and recall
    f1_scores = [2 * p * r / (p + r) if (p + r) > 0 else 0.0 for p, r in zip(precisions, recalls)]
    avg_f1 = sum(f1_scores) / len(f1_scores)

    # Accuracy = proportion of non-zero matches (partial or full)
    match_count = sum((1 if (p > 0 and r > 0) else 0) for p, r in zip(precisions, recalls))
    accuracy = match_count / len(merged_df)

    results[col] = {
        'Precision (%)': round(avg_precision * 100, 2),
        'Recall (%)': round(avg_recall * 100, 2),
        'F1 Score (%)': round(avg_f1 * 100, 2),
        'Accuracy (%)': round(accuracy * 100, 2)
    }

# Print results
for col, metrics in results.items():
    print(f"{col}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")
    print()

Sellers:
  Precision (%): 65.6
  Recall (%): 67.83
  F1 Score (%): 62.33
  Accuracy (%): 86.76

Buyers:
  Precision (%): 60.72
  Recall (%): 65.72
  F1 Score (%): 59.73
  Accuracy (%): 76.47

Brokers:
  Precision (%): 61.13
  Recall (%): 62.21
  F1 Score (%): 58.47
  Accuracy (%): 73.53

Surgery:
  Precision (%): 73.77
  Recall (%): 76.27
  F1 Score (%): 72.6
  Accuracy (%): 84.56

